<a href="https://colab.research.google.com/github/Hanbin-git/Dacon_cacer/blob/main/%EC%84%B1%EB%8A%A5%EB%96%A8%EC%96%B4%EC%A7%84%20%EC%A0%84%EB%9E%B5%EC%9C%BC%EB%A1%9C%20%ED%99%95%EC%9D%B8%EB%90%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip -o "/content/drive/MyDrive/open_1.zip" -d "/content/open_1"


Archive:  /content/drive/MyDrive/open_1.zip
  inflating: /content/open_1/sample_submission.csv  
  inflating: /content/open_1/test.csv  
  inflating: /content/open_1/train.csv  


In [3]:
import os

def get_path(filename):
    return "/content/open_1/" + filename


In [4]:
# SMOTE 설치
!pip install -U imbalanced-learn
!pip install lightgbm catboost optuna



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 22.2 MB/s eta 0:00:00


In [5]:
!pip install xgboost lightgbm


In [6]:
import pandas as pd
import numpy as np
import os

# 경로 함수 정의
def get_path(filename):
    return "/content/open_1/" + filename

# 데이터 로딩
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))

# target 분리
y = train['Cancer']
X = train.drop(columns=['ID', 'Cancer'])
X_test = test.drop(columns=['ID'])


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# 경로 함수
def get_path(filename):
    return "/content/open_1/" + filename

# 데이터 로드
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

# Target 분리
y = train['Cancer']
X = train.drop(columns=['ID', 'Cancer'])
X_test = test.drop(columns=['ID'])

# ✅ 파생변수 생성 함수
def add_derived_features(df):
    df = df.copy()
    num_cols = df.select_dtypes(include='number').columns.tolist()

    for col in num_cols:
        df[f'{col}_squared'] = df[col] ** 2
        df[f'{col}_sqrt'] = np.sqrt(np.abs(df[col]))
        df[f'{col}_log'] = np.log1p(np.abs(df[col]))

    if 'TSH' in df.columns and 'T4' in df.columns:
        df['T4_TSH_ratio'] = df['T4'] / (df['TSH'] + 1e-3)

    if 'T3' in df.columns and 'Nodule_Size' in df.columns:
        df['T3_times_Nodule'] = df['T3'] * df['Nodule_Size']

    return df

X = add_derived_features(X)
X_test = add_derived_features(X_test)

# ✅ 전처리 함수 (문자형: Label Encoding, 수치형: 평균 대체)
def preprocess(df, fit_encoders=None):
    df = df.copy()
    encoders = fit_encoders if fit_encoders else {}
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy='mean').fit_transform(df[[col]])
    return df, encoders

X, encoders = preprocess(X)
X_test, _ = preprocess(X_test, encoders)

# ✅ 5-Fold 학습 및 앙상블 예측
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))

for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
    X_tr, y_tr = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[valid_idx], y.iloc[valid_idx]

    model1 = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=fold)
    model2 = LGBMClassifier(random_state=fold)
    model3 = CatBoostClassifier(verbose=0, random_state=fold)

    ensemble_model = VotingClassifier(
        estimators=[('xgb', model1), ('lgbm', model2), ('cat', model3)],
        voting='soft'
    )
    ensemble_model.fit(X_tr, y_tr)

    oof_preds[valid_idx] = ensemble_model.predict_proba(X_val)[:, 1]
    test_preds += ensemble_model.predict_proba(X_test)[:, 1] / kf.n_splits

# ✅ Threshold 최적화
thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"Best F1: {max(f1s):.4f} at threshold {best_thresh:.2f}")

# ✅ 결과 저장
submission['Cancer'] = (test_preds > best_thresh).astype(int)
submission.to_csv("submission_final.csv", index=False)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [05:05:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014317 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4405
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [05:05:46] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011341 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4405
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [05:06:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012942 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4401
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [05:06:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011396 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4405
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [05:07:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 8368, number of negative: 61360
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011183 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4401
[LightGBM] [Info] Number of data points in the train set: 69728, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120009 -> initscore=-1.992343
[LightGBM] [Info] Start training from score -1.992343
Best F1: 0.4860 at threshold 0.22


In [8]:
from google.colab import files
files.download("submission_final.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
import torch
print("CUDA available:", torch.cuda.is_available())


CUDA available: True


In [10]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score
import joblib  # 모델 저장용

# ✅ F1 score 평가 함수 (for Optuna)
def f1_cv(model, X, y):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    return cross_val_score(model, X, y, scoring=make_scorer(f1_score), cv=skf).mean()

# ✅ Optuna objective 함수
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 800),
        "max_depth": trial.suggest_int("max_depth", 4, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4),
    }

    model = RandomForestClassifier(
        **params,
        random_state=42,
        n_jobs=-1,
        verbose=0
    )

    return f1_cv(model, X, y)

# ✅ Optuna Study 구성
study = optuna.create_study(direction='maximize', study_name="RF_F1", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=30, timeout=600)  # 최대 10분 제한

# ✅ 최적 하이퍼파라미터 출력 및 저장
print("Best trial:")
print("  Value: {:.4f}".format(study.best_trial.value))
print("  Params: ", study.best_trial.params)

# ✅ 모델 학습 및 저장
best_model = RandomForestClassifier(
    **study.best_trial.params,
    random_state=42,
    n_jobs=-1
)
best_model.fit(X, y)

joblib.dump(best_model, "best_rf_model.pkl")


[I 2025-06-17 05:07:36,124] A new study created in memory with name: RF_F1
[I 2025-06-17 05:16:20,342] Trial 0 finished with value: 0.11346833971754519 and parameters: {'n_estimators': 362, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.11346833971754519.
[I 2025-06-17 05:18:21,661] Trial 1 finished with value: 0.0 and parameters: {'n_estimators': 209, 'max_depth': 6, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.11346833971754519.


Best trial:
  Value: 0.1135
  Params:  {'n_estimators': 362, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3}


['best_rf_model.pkl']

In [11]:
# 최적 하이퍼파라미터 추출
best_params = study.best_trial.params

# 최적 모델 학습
best_model = RandomForestClassifier(
    **best_params, random_state=42, n_jobs=-1
)
best_model.fit(X, y)

# 테스트셋 예측
test_pred_probs = best_model.predict_proba(X_test)[:, 1]

# 임계값 최적화 (훈련 데이터 기반)
thresholds = np.arange(0.1, 0.9, 0.01)
oof_preds = cross_val_score(best_model, X, y, scoring='f1', cv=5)
print(f"Cross-validated F1: {oof_preds.mean():.4f}")

# 기본 임계값 0.5 적용해 제출 파일 생성
submission['Cancer'] = (test_pred_probs > 0.5).astype(int)
submission.to_csv("submission_optuna_rf.csv", index=False)


Cross-validated F1: 0.1077


In [12]:
from google.colab import files
files.download("submission_optuna_rf.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>